In [34]:
import geopandas as gpd
import numpy as np

In [55]:
shp = gpd.read_file("../../annotations/new/Raqqa/Damage_Sites_Raqqa_CDA.shp")

In [56]:
shp.tail(), shp.columns

(                             SiteID SensDt SensId Confidence DaSitCl  \
 13155  Building (General / Default)   None   None       None    None   
 13156  Building (General / Default)   None   None       None    None   
 13157  Building (General / Default)   None   None       None    None   
 13158  Building (General / Default)   None   None       None    None   
 13159  Building (General / Default)   None   None       None    None   
 
       SensorDate SensID2 ConfID2 DaSitCl2 Damst2  ...    ConfId5  \
 13155       None    None    None     None   None  ...  Very High   
 13156       None    None    None     None   None  ...     Medium   
 13157       None    None    None     None   None  ...     Medium   
 13158       None    None    None     None   None  ...     Medium   
 13159       None    None    None     None   None  ...  Very High   
 
               DaSitCl5        Damst5             GrDaCl  \
 13155        Destroyed  New - damage  Damaged Buildings   
 13156  Moderate Damage 

In [57]:
tcols = [c for c in shp.columns if 'SensDt' in c]
dcols = [c for c in shp.columns if 'DaSitCl' in c and c !="DaSitCl"]

In [58]:
tcols, dcols

(['SensDt', 'SensDt3', 'SensDt4', 'SensDt5'],
 ['DaSitCl2', 'DaSitCl3', 'DaSitCl4', 'DaSitCl5'])

In [59]:
for i, c in enumerate(tcols):
    if i==0:
        allDates = shp[c]
    else:
        allDates = allDates.append(shp[c])

allDates        
sensor_date_values = allDates.unique()
sensor_date_values = sensor_date_values[sensor_date_values != np.array(None)]
sensor_date_values

/var/folders/z1/7pydgng971nfzdf2rwg_p__r0000gn/T/ipykernel_2314/21167257.py:5: FutureWarning: The series.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  allDates = allDates.append(shp[c])
/var/folders/z1/7pydgng971nfzdf2rwg_p__r0000gn/T/ipykernel_2314/21167257.py:5: FutureWarning: The series.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  allDates = allDates.append(shp[c])


array(['2013-10-22', '2015-05-29', '2017-02-03', '2017-10-21'],
      dtype=object)

In [60]:
new_df = []
for i, row in shp.iterrows():

    row_entry = {}
    row_entry['geometry'] = row['geometry']
    
    for j, sensor_date_col in enumerate(tcols):
        if(row[sensor_date_col] != None):
            row_entry[row[sensor_date_col]] = row[dcols[j]]
    
    new_df.append(row_entry)

In [61]:
df = gpd.GeoDataFrame(new_df)
df = df.replace({np.nan: np.nan, "Destroyed": 3, "Severe Damage": 2, "Moderate Damage": 1, "No Visible Damage": 0, "Impact Crater (Damage to Road)": np.nan, 'Impact Crater (Damage to Field)': np.nan, "6": np.nan})
df = df.set_crs("epsg:4326")

In [62]:
df.isna().sum()

geometry          0
2013-10-22    12820
2015-05-29    11592
2017-02-03    11342
2017-10-21      262
dtype: int64

In [63]:
df.to_file("raqqa_damage.gpkg", driver="GPKG")